# Parameter Estimation with Expectation-Maximization (EM)

The Expectation-Maximization (EM) algorithm is a method for estimating unknown parameters in probabilistic models when some variables are unobserved. In the context of Kalman Filters, we will use EM to estimate Q and R from data, without knowing the true underlying states.

This is particularly valuable because in real-world applications, we often don't know the true noise characteristics of our system, but we can learn them from the data itself.


## Goal

In a Kalman Filter, we need to specify:
- Q matrix: Process noise covariance - how much uncertainty we expect in the system dynamics.
- R matrix: Observation noise covariance - how much uncertainty we expect in our measurements

These parameters significantly affect the filter's performance, but they're often unknown in practice. By using the EM algorithm, we can have a good estimation of them from the data.
We must note that in order to estimate them properly, we need data with the same underlying real process noise and observation noise.

## Brief Introduction To Smoothing
We already talked briefly about smoothing in notebook 2. However, we would go a little deeper in the technical details in order to understand better how the EM works in our case of the KF.

Specifically, we would talk about a smoothing algorithm called RTS (Rauch-Tung-Striebel).

The RTS smoother works by:
1. Running the Kalman Filter forward to get filtered estimates of the states and covariances (both a-preori and a-postiriori).
2. Running a backward pass to incorporate future information into past filtered estimates.

The forward pass is the basic predict -> update loop that we already covered in the KF.

The backward pass uses:
$$\hat{x}_{k|T} = \hat{x}_{k|k} + J_k (\hat{x}_{k+1|T} - \hat{x}_{k+1|k})$$

where $J_k$ is the smoothing gain that tells us how much to trust the future information.
The smoothing gain $J_k$ is computed as:
$$J_k = P_{k|k} F^T P_{k+1|k}^{-1}$$

(We already know this notation from notebook 2)
The smoothing gain tells us how much to adjust our current estimate $\hat{x}_{k|k}$ based on future information (The a-posteriori covariance matrix of the next state). It's proportional to:
- How uncertain we were about the state at time $k$ ($P_{k|k}$)
- How much the state at time $k$ affects the state at time $k+1$ ($F^T$)
- How much we trust our prediction at time $k+1$ ($P_{k+1|k}^{-1}$)

The intuition is that if $J_k$ is large, it means that the next state has extra information about $x_k$ than what we already know, so by taking the information that we got from smoothing, we would get a better estimation for $x_k$

## How EM Works

The EM algorithm is an iterative method that alternates between two steps:

## E-step (Expectation)
Given current estimates of Q and R, we compute the expected values of the hidden states.
Intuitively, we will get the most accurate expected values of them, if we will use all the observed data.
Meaning, given all the observed data, what would we expect the hidden states to be. 
This is exactly what smoothing does (We mentioned it briefly in notebook 2).

Given measurements $y_1, y_2, \ldots, y_T$, we want to estimate the hidden state $x_k$ at time $k$ using all the measurements.

The smoothed state estimate is:
$$\hat{x}_{k|T} = E[x_k | y_1, y_2, \ldots, y_T]$$

And the smoothed covariance is:
$$P_{k|T} = \text{Cov}[x_k | y_1, y_2, \ldots, y_T]$$

In the EM algorithm, we need to compute expectations like:
- $E[x_k | y_1, \ldots, y_T]$ (the smoothed state)
- $E[x_k x_k^T | y_1, \ldots, y_T]$ (for covariances)
- $E[x_k x_{k-1}^T | y_1, \ldots, y_T]$ (for pairwise covariances)

These are exactly what the RTS smoother computes. The smoother gives us:
- $\hat{x}_{k|T}$ = $E[x_k | y_1, \ldots, y_T]$
- $P_{k|T}$ = $E[x_k x_k^T | y_1, \ldots, y_T] - \hat{x}_{k|T} \hat{x}_{k|T}^T$
- $V_{k,k-1}$ = $E[x_k x_{k-1}^T | y_1, \ldots, y_T] - \hat{x}_{k|T} \hat{x}_{k-1|T}^T$


## M-step (Maximization)  
Given the expected values, we update Q and R to maximize the likelihood of the observed data.

#### R Matrix Update
The R matrix represents the uncertainty in our measurements. We update it using (from the paper https://www.stat.pitt.edu/stoffer/dss_files/em.pdf):
$$R_{\text{new}} = \frac{1}{T} \sum_{t=1}^T \left[ (y_t - H \hat{x}_{t|T})(y_t - H \hat{x}_{t|T})^T + H P_{t|T} H^T \right]$$

#### Q Matrix Update (Process Noise)
The Q matrix represents the uncertainty in our system dynamics. We update it using:

$$Q_{\text{new}} = \frac{1}{T-1} \sum_{t=1}^{T-1} \left[ (\hat{x}_{t+1|T} - F \hat{x}_{t|T})(\hat{x}_{t+1|T} - F \hat{x}_{t|T})^T + F P_{t|T} F^T + P_{t+1|T} - V_{t+1,t} F^T - F V_{t+1,t}^T \right]$$


The algorithm iterates between these steps until convergence.

<br>

## Step-Size For Updates
In the empirical tests I did, I saw that there could be rather big jumps of values when estimating R and Q.
I saw that I got better estimations when I calculated R and Q while incorporating "steps" towards the current calculated values. Meaning, after an iteration of the EM, instead of the assignment above of $Q_{\text{new}}, R_{\text{new}}$, I updated them with some average between the newly calculated and the old ones.